In [1]:
import logging
import pandas as pd
import multiprocessing as mp

from remediator import get_remediation

logging.basicConfig(level=logging.ERROR)

In [2]:
incompats = pd.read_csv("res/license_incompatibilities.csv")
incompat_set = set(zip(incompats.package, incompats.version))
mongo_uri = "mongodb://localhost:27017/"

# for pkg, ver in sorted(incompat_set):
#     print(pkg, ver)
#     print(get_remediation(mongo_uri, pkg, ver)["changes"])

def worker(mongo_uri, pkg, ver):
    try:
        return get_remediation(mongo_uri, pkg, ver)
    except Exception as ex:
        return {"package": pkg, "version": ver, "error": str(ex)}

params = [(mongo_uri, pkg, ver) for pkg, ver in incompat_set]
with mp.Pool(mp.cpu_count() // 2) as pool:
    result = pool.starmap(worker, params)
result = pd.DataFrame(result).sort_values(by=["package", "version"])
result.to_csv("res/remediation.csv", index=False)
result.head()

,package,version,changes,new_tree,error
138,amundsen-databuilder,1.5.0,[Update unidecode from 1.1.1 to 0.04.6],"{'pytest-mock': '1.13.0', 'statsd': '3.2.1', '...",NaN
24,amundsen-databuilder,4.0.3,[Update unidecode from 1.1.1 to 0.04.6],"{'amundsen-databuilder': '4.0.3', 'statsd': '3...",NaN
133,amundsen-databuilder,6.4.1,"[Update amundsen-rds from 0.0.6 to 0.0.2, Upda...","{'statsd': '3.3.0', 'marshmallow': '3.14.1', '...",NaN
18,amundsen-databuilder,7.4.2,[Update unidecode from 1.3.6 to 0.04.6],"{'jsonref': '0.2', 'statsd': '3.3.0', 'python-...",NaN
29,ansible-lint,4.0.0,[Remove ansible],"{'six': '1.12.0', 'ansible-lint': '4.0.0', 'py...",NaN
